In [1]:


import numpy as np
from helper import NonConvexDecomposition
import polytope  

### Velocity constraints
vx_max = 0.05  # Maximum velocity in x-direction (m/s)
vy_max = 0.05  # Maximum velocity in y-direction (m/s)
vel_vertices = np.array([[vx_max, vy_max], [vx_max, -vy_max], [-vx_max, -vy_max], [-vx_max, vy_max]])


#### Acceleration constraints
ax_max = 1.0  # Maximum acceleration in x-direction (m/s^2)
ay_max = 1.0  # Maximum acceleration in y-direction (m/s^2)
accel_vertices = np.array([[ax_max, ay_max], [ax_max, -ay_max], [-ax_max, -ay_max], [-ax_max, ay_max]])


### In X-Y coordinate ###
Workspace = np.array(([[178,  47],
    [704,  47],
    [704, 817],
    [178, 817]]))


## Convex Unsafe Region: Like vertices 
# Obs1 =  [array([[568, 429],
Obs1 = np.array([[568, 429],
                [703, 429],
                [703, 816],
                [568, 816]])

Obs2 = np.array([[308, 181],
                [438, 181],
                [438, 691],
                [308, 691]]) 

Obs3 = np.array([[437,  47],
                [568,  47],
                [568, 309],
                [437, 309]])

# Convex Inequality Constraints: Ax <= b
w_poly = poly = polytope.qhull(Workspace)
Obs = [Obs1, Obs2, Obs3]
obs_poly = [polytope.qhull(box) for box in Obs]
vel_poly = polytope.qhull(vel_vertices)
accel_poly = polytope.qhull(accel_vertices)

wA = w_poly.A
wb = w_poly.b

## velocity convex constraint
## 0, 0 at x-y coordinate only vx, vy is constrained.
F_v = np.hstack([np.zeros((vel_poly.A.shape[0], 2)), vel_poly.A])
f_v = vel_poly.b

# Sate Convex Inequality Constraints: Ax <= b
F_w = np.hstack([wA, np.zeros((wA.shape[0], 2))])
f_w = wb

# Obstacle unsafe region
## For each obstacle polygon convex unsafe set in x-y coor, 0, 0, added for velocity
F_obs = []
for obs in obs_poly:
    F = np.hstack([obs.A, np.zeros((obs.A.shape[0], 2))])
    fvec = obs.b
    F_obs.append((F, fvec))

safe_state_sets = [
(F_w, f_w),
(F_v, f_v),
]

# 2) Unsafe sets (inside of obstacles) as (A, b) with A x <= b
unsafe_state_sets = []
for obs in F_obs:
    unsafe_state_sets.append((np.asarray(obs[0], dtype=float), np.asarray(obs[1], dtype=float)))

# 3) Build facet-wise disjunctions with an optional margin (eps)
nonconvex_decomp = NonConvexDecomposition(eps=0.0)
Fx_list, fx_list = nonconvex_decomp.build(safe_state_sets, unsafe_state_sets)

Gu = accel_poly.A
gu = accel_poly.b

# State transition matrix with damping
dt = 0.1; damping_fun = 1 - 0.1 * dt
A = np.array([[1, 0, dt, 0],
                   [0, 1, 0, dt],
                   [0, 0, damping_fun, 0],
                   [0, 0, 0, damping_fun]])

B = np.array([[0.5 * dt**2, 0],
           [0, 0.5 * dt**2],
           [dt, 0],
           [0, dt]])


E = np.array([[1., 0., 0., 0.],
                           [0., 1., 0., 0.],
                           [0., 0., 1., 0.],
                           [0., 0., 0., 1.]])

Gw = np.array([[ 1.,  0.,  0.,  0.],
                             [ 0.,  1.,  0.,  0.],
                             [ 0.,  0.,  1.,  0.],
                             [ 0.,  0.,  0.,  1.],
                             [-1., -0., -0., -0.],
                             [-0., -1., -0., -0.],
                             [-0., -0., -1., -0.],
                             [-0., -0., -0., -1.]])
        
Fw = np.array([[0.00220696731294516, 0.00033361141907932, 0.01511132840251176, 0.00392036854057386, 0.00137375336326578, 0.00060107734896421, 0.01921057797980761, 0.00721068247405685]]).T
      

In [2]:
import sys, os
from pathlib import Path

sys.path.insert(0, '../')
import config1
cfg = config1.config()

project_root /home/shivam/Aalto Dropbox/Shivam Chaubey/research_work/codes/assistive_system/dev/misc


In [3]:
from CIS_computation import ExtractCIS
cis_builder = ExtractCIS(cfg, Fx_list = Fx_list, fx_list = fx_list,
                 Gu = Gu, gu = gu, E = E, Gw = Gw, 
                 Fw = Fw, use_matlab = False, safe_set_path = None)

allMatlabIds []


SyntaxError: unexpected EOF while parsing (2193774569.py, line 6)

In [ ]:


T_list = np.vstack([F, Fx])
t_list = np.hstack([fvec, fxvec])


In [ ]:
F

array([-1., -0.,  0.,  0.])